<a href="https://colab.research.google.com/github/IanGZC/Diplomado-/blob/main/Data_Analyst_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Analyst Assignment

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
from datetime import datetime, date, time
import numpy as np
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/ My \Drive/Diplomado

d1=pd.read_excel('Assignment_2.xlsx', sheet_name='Employee_Data')
d2=pd.read_excel('Assignment_2.xlsx', sheet_name='Completed_Trainings')
d3=pd.read_excel('Assignment_2.xlsx', sheet_name='Performance Data')

Mounted at /content/drive
/content/drive/ My Drive/Diplomado


In [ ]:
d4=d1.merge(d2, on = ['Employee_ID'], how = 'outer')
d5=d4.merge(d3, on = ['Employee_ID'], how = 'outer')
d5=d5.drop_duplicates(subset = ['Employee_ID'])
d5=d5[d5.Employee_ID >= 1]
d5

,Employee_ID,SVP Leader,Business Title,Cost Center,Cost Center Family,Length of service,Leave Status,is People Manager?,Region,Manager IC Helper,...,Type,Stage 2+ Date,Stage,Close Date,Product Rate Plan Charge,Product Name,Add-On ARR (converted) Currency,Add-On ARR (converted),Total Commissionable ARR (converted) Currency,Total Commissionable ARR (converted)
0,1.0,Leader 1,Senior Commercial Account Executive,532 Commercial AE,Commercial,20.0,Active,False,EMEA,IC,...,Expansion,2017-11-10,06 - Signed,2018-01-07,Suite Subscription,Suite,USD,0.0,USD,563863.0
5,2.0,Leader 1,Senior Commercial Account Executive,532 Commercial AE,Commercial,13.0,Active,False,EMEA,IC,...,Expansion,2017-08-11,04 - Solution Validation,2018-02-23,Suite Subscription,Support: Addons,USD,21495.0,USD,13924.0
9,3.0,Leader 1,Senior Commercial Account Executive,552 Mid-Market AE,Mid-Market,44.0,Active,False,EMEA,IC,...,Expansion,2017-12-21,06 - Signed,2018-01-07,Suite Subscription,Suite,USD,0.0,USD,177562.0
11,4.0,Leader 2,Enterprise Corporate Account Executive,508 LATAM Enterprise AE,Enterprise,9.0,Active,False,LATAM,IC,...,New Business,2017-12-09,06 - Signed,2018-01-17,Suite Subscription,Suite,USD,0.0,USD,70689.0
14,5.0,Leader 1,Senior Commercial Account Executive,532 Commercial AE,Commercial,15.0,Active,False,EMEA,IC,...,Expansion,2017-09-21,02 - Discovery,2018-03-23,Suite Subscription,Suite,USD,532545.0,USD,206668.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3444,2528.0,Leader 5,Contingent Worker,256 Prof Services Ops,Services,0.0,Active,False,EMEA,IC,...,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3445,2529.0,Leader 5,Contingent Worker,256 Prof Services Ops,Services,0.0,Active,False,EMEA,IC,...,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3446,2530.0,Leader 5,Contingent Worker,256 Prof Services Ops,Services,0.0,Active,False,EMEA,IC,...,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3447,2531.0,Leader 5,Contingent Worker,256 Prof Services Ops,Services,0.0,Active,False,EMEA,IC,...,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
## 1. What is the training completion rate for each course by segments (SVP Leader/Region), factoring in the following caveats?
#     Training is required for all employees except:

#     -Employees currently on leave are exempt from the training requirement.
#     -'Sell More Suite SKU' course is not required for employees within the 'Advocacy' cost center family.
#     -“Suite/Automation Technical Lab” and “Advanced Suite Bots Lab Course” courses are required only for employees in the 'PreSales' and 'Services' cost center family.

q1=d4[['Employee_ID', 'SVP Leader', 'Region', 'Training_Completed', 'Leave Status', 'Cost Center Family']]

#Primero se eliminan de la tabla todos los datos de los trabajadores cuyo estatus sea "On Leave"
q1=q1.loc[q1['Leave Status'] == 'Active']
q1 = q1.drop(columns = ['Leave Status'])

q1=pd.DataFrame(q1.groupby(['Region', 'Cost Center Family', 'SVP Leader', 'Training_Completed', 'Employee_ID']).sum()).reset_index()
q1=q1.sort_values(['Cost Center Family'])

#Se crea un ciclo que contenga un condicional para que en caso de que estén presentes las dos variables indeseadas en una columna, se haga una fila vacía y posteriormente se elimine
for i in range(1741):
  if (q1['Cost Center Family'].loc[i]=='Advocacy' and q1['Training_Completed'].loc[i]=='Sell More Suite SKU'):
    q1.loc[i]=None
q1.dropna()

#Se grafica respecto a la región

q1=pd.DataFrame(q1.groupby(['Region', 'Training_Completed', 'Employee_ID']).value_counts().reset_index().rename(columns = {0:'Number of Employee'}))
q1
fig = px.bar(q1, x='Training_Completed', y='Number of Employee', color='Region', facet_col="Region", facet_col_wrap=4)
fig.show()

In [ ]:
## 2. How would you analyze the performance of an account executive? Is there a difference
#     between those who have completed training and those who have not? How would you segment
#     the data and present your findings to senior stakeholders?
#
#     Hint: Look at this from an overall employee perspective who has completed any training vs.
#     those who have not completed any training. Any training would count,rather than distinguishing
#     which specific courses were completed, as they all contribute to one Suite product.

q2=d5[['Employee_ID', 'Business Title','Training_Completed', 'Add-On ARR (converted)', 'Total Commissionable ARR (converted)']]


#Se filtra la tabla para solo tener a los ejecutivos de venta

fil = q2[q2["Business Title"].str.contains('|'.join(["Account", "Executive"]))]
fil['Gains']=fil['Add-On ARR (converted)']+fil['Total Commissionable ARR (converted)']
fil['Employee_ID'] = fil['Employee_ID'].astype(str)

#Se asume que el performance de un ejecutivo de cuenta está en función de las ganancias

fig = px.bar(fil, x='Employee_ID', y='Gains')
fig.show()

#Se crea una columna extra que contenga un booleano en caso de haber entrenamiento

fil['Training']=np.NaN
fil['Training']=fil['Training_Completed'].notnull()

g=pd.DataFrame(fil.groupby(['Training']).Gains.sum().reset_index())


fig = px.bar(g, y='Gains', x='Training')
fig.show()

<ipython-input-291-bfc4ba826058>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-291-bfc4ba826058>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<ipython-input-291-bfc4ba826058>:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-291-bfc4ba826058>:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



3. Any other interesting insights that you can see in the data? Any data quality issues with the data?
Any challenges with analyzing the data? What additional data do you think would be useful for further analyzing the existing datasets?

Hay muchas columnas que para esta tarea en concreto no fueron muy útiles. La información está muy desorganizada y contiene muchos elementos nulos.

## Data Challenge

Sales leadership has tasked you with analyzing the data to discover how employee training
impacts sales performance and effectiveness. The goal is to identify patterns and provide
actionable recommendations that drive sales growth and improve team performance by
answering the following questions.
Questions
1. What is the training completion rate for each course by segments (SVP Leader/Region),
factoring in the following caveats? Training is required for all employees except:
* Employees currently on leave are exempt from the training requirement.
* 'Sell More Suite SKU' course is not required for employees within the 'Advocacy'
cost center family.
* “Suite/Automation Technical Lab” and “Advanced Suite Bots Lab Course”
courses are required only for employees in the 'PreSales' and 'Services' cost
center family.

2. How would you analyze the performance of an account executive? Is there a difference
between those who have completed training and those who have not? How would you
segment the data and present your findings to senior stakeholders?
* Hint: Look at this from an overall employee perspective who has completed any
training vs. those who have not completed any training. Any training would count,

rather than distinguishing which specific courses were completed, as they all
contribute to one Suite product.

3. Any other interesting insights that you can see in the data? Any data quality issues with
the data? Any challenges with analyzing the data? What additional data do you think
would be useful for further analyzing the existing datasets?



---

## Data Dictionary

### Table 1 : Employee Information

This table maintains critical employment details for organization members, including
identification, job titles, management hierarchies, tenure, and work region

* **Employee_ID:** Unique identifier for each employee.
* **SVP Leader:** Leader overseeing the employee.
* **Business Title:** Official job title of the employee.
* **Cost Center:** Identifier for the employee's department or unit for cost tracking.
* **Cost Center Family:** Group of related cost centers for financial reporting.
* **Length of Service:** Total time the employee has worked at the company.
* **Leave Status:** Indication of whether the employee is on leave.
* **Is People Manager?:** Indicates if the employee oversees other staff.
* **Region:** Geographic area where the employee works.
* **Manager IC Helper:** Additional data supporting managerial status.
* **IC:** Individual Contributor

### Table 2: Completed Trainings
This table records the professional development activities of employees by linking completed
training programs to their unique identifiers
* **Employee_ID:** Unique identifier linked to an employee who completed the training.
* **Training Name:** Name of the training program or course that the employee completed.

### Table 3: Performance Data
This table includes the sales and revenue generation activities of employees by cataloguing
opportunities, their progression, and financial outcomes. It captures granular data on sales
stage milestones, product-related charges, and revenue figures, all linked by employee and
opportunity identifiers, making it a vital asset for analyzing sales performance and compensation
metrics.

* **Employee_ID:** Unique identifier for the employee associated.
* **Opportunity ID:** Unique identifier for the sales opportunity.
* **Type:** The category or classification of the opportunity.
* Expansion is for existing business

* **Stage 2+ Date:** The date when the opportunity reached or surpassed stage 2 in the
sales process.
* **Stage:** Current stage of the opportunity in the sales pipeline.
 * 02 - Discovery: Initial stage where potential needs and opportunities are
identified with the client.
 * 03 - Solution Review: Potential solutions are presented and reviewed with the
client.
 * 04 - Solution Validation: Client feedback is incorporated, and solutions are
refined and validated.
 * 05 - Contracting / Verba: Terms are negotiated and a verbal agreement may be
reached.
 * 06 - Signed/07 - Closed: Formal agreement is executed with signatures from all
parties. Signed and Closed are counted as finalized
* **Close Date:** The date when the opportunity was closed.
* **Product Rate Plan Charge:** The charge associated with the product's rate plan.
* **Product Name:** The name of the product related to the opportunity.
* **Add-On ARR (converted):** The value of the additional ARR from add-ons,
* **Total Commissionable ARR (converted):** The total annual recurring revenue that is
eligible for commission